In [8]:
import pandas as pd
import numpy as np
import warnings

In [10]:
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("D:/DataIITM/sourcelocalization.csv")

In [3]:
data.head()

,Unnamed: 0,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15625000 entries, 0 to 15624999
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   rec_x       float64
 2   rec_y       float64
 3   rec_z       float64
 4   roll_r      float64
 5   pitch_r     float64
 6   yaw_r       float64
 7   mag_x       float64
 8   mag_y       float64
 9   mag_z       float64
 10  target_x    float64
 11  target_y    float64
 12  target_z    float64
dtypes: float64(12), int64(1)
memory usage: 1.5 GB


In [5]:
data.drop("Unnamed: 0" , axis =1, inplace=True)

In [6]:
data.head()

,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [44]:
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split

In [50]:
from tensorflow import keras 
from keras.models import Sequential 
from keras.layers import Dense , Dropout , BatchNormalization , Input
from keras.losses import mean_squared_error
from keras.metrics import R2Score 
from keras.utils import plot_model

### Splitting The Data into Training , Testing , Validating Sets

In [14]:
features = data.drop(columns=["target_x" , "target_y" , "target_z"])
targets = data[["target_x","target_y","target_z"]]

In [17]:
train_features , test_features , train_targets , test_targets = train_test_split(features, targets , train_size=0.7 , random_state=42)

In [18]:
train_features , val_features , train_targets , val_targets = train_test_split(train_features , train_targets , train_size=0.8 , random_state=42)

In [19]:
print("Total Training Points : ",len(train_features))
print("Total validation Points : ",len(val_features))
print("Total Testing Points : ",len(test_features))

Total Training Points :  8750000
Total validation Points :  2187500
Total Testing Points :  4687500


## Scaling of the Data using Transformers

In [23]:
tf = ColumnTransformer(
    [
        ("Scaling" , StandardScaler() , slice(0,10))
    ]
)

In [39]:
train_f = tf.fit_transform(train_features)

In [42]:
val_f = tf.transform(val_features)
test_f = tf.transform(test_features)

## Model Creation 

In [47]:
model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(9 , activation= 'linear'))
model.add(Dense(16 , activation= 'selu'))
model.add(Dense(24 , activation= 'linear'))
model.add(Dense(56 , activation= 'selu'))
model.add(Dense(24 , activation= 'linear'))
model.add(Dense(16 , activation= 'selu'))
model.add(Dense(8 , activation= 'linear'))
model.add(Dense(3 , activation='linear'))

In [48]:
model.compile(loss = mean_squared_error , metrics=[R2Score()])

In [49]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 9)              │            90 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 56)             │         1,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 24)             │         1,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,989 (15.58 KB)

 Trainable params: 3,989 (15.58 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
plot_model(model ,to_file='model.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


## Training and Validating the Model

In [56]:
history = model.fit(train_f , train_targets.values , epochs=5, validation_data=[val_f , val_targets.values])

Epoch 1/5
273432/273438 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1026.0641 - r2_score: 0.3004